In [162]:
import pandas as pd 
from tensorflow import keras
from keras.utils import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical, pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [163]:
import pandas as pd

df = pd.read_parquet("hf://datasets/gabrielmbmb/text-generation-distilabel-ray/data/train-00000-of-00001.parquet")

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   instruction          327 non-null    object
 1   completion           327 non-null    object
 2   meta                 327 non-null    object
 3   generation           327 non-null    object
 4   distilabel_metadata  327 non-null    object
 5   model_name           327 non-null    object
dtypes: object(6)
memory usage: 15.5+ KB


In [165]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary resources
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to preprocess individual texts (now working directly on strings)
def preprocess_text(text):
    text = text.lower()  # Lowercase the text
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])  # Remove punctuation
    tokens = text.split()  # Tokenize by splitting
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize the tokens
    return ' '.join(tokens)  # Return the cleaned and lemmatized text

# Apply the preprocessing function directly to the 'completion' column
df['processed_messages'] = df['completion'].apply(preprocess_text)

# Check the results
print(df[['completion', 'processed_messages']].head())  # Display the processed messages


                                          completion  \
0  Denote the number of chocolates each person ha...   
1  Elon Musk hires a team of experts to build the...   
2  Clerk: How are you doing today?\nCustomer: Gre...   
3  The sun and the moon, the guards from the sky\...   
4  No, Searle does not believe that AI can think....   

                                  processed_messages  
0  denote number chocolate person letter first na...  
1  elon musk hire team expert build ultimate yach...  
2  clerk today customer great im buying grocery c...  
3  sun moon guard sky one work day watch night ra...  
4  searle believe ai think step step explanation ...  


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\92317\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\92317\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [166]:
# Add this to verify that 'completion' has the correct structure
print(df['completion'].apply(lambda x: type(x)).value_counts())  # Check types of data in 'completion'
print(df['completion'].head(1))  # View one example of 'completion'

completion
<class 'str'>    327
Name: count, dtype: int64
0    Denote the number of chocolates each person ha...
Name: completion, dtype: object


In [167]:
print(df['processed_messages'].head())

0    denote number chocolate person letter first na...
1    elon musk hire team expert build ultimate yach...
2    clerk today customer great im buying grocery c...
3    sun moon guard sky one work day watch night ra...
4    searle believe ai think step step explanation ...
Name: processed_messages, dtype: object


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   instruction          327 non-null    object
 1   completion           327 non-null    object
 2   meta                 327 non-null    object
 3   generation           327 non-null    object
 4   distilabel_metadata  327 non-null    object
 5   model_name           327 non-null    object
 6   processed_messages   327 non-null    object
dtypes: object(7)
memory usage: 18.0+ KB


In [169]:
# Combine all processed conversations into a single string
all_processed_text = ' '.join(df['processed_messages'])

# Split the combined text into individual words
words = all_processed_text.split()

# Use a set to find unique words
unique_words = set(words)

# Find the total number of unique words
total_unique_words = len(unique_words)

# Print the result
print(f"Total unique words: {total_unique_words}")

# Optionally, print the unique words
print(unique_words)


Total unique words: 4215
{'charise', 'planner', 'yard', 'paragraph', 'interference', 'human', '47', 'corporate', 'capitola', 'cheer', 'dazzling', 'lower', 'desert', 'doubledigit', 'applied', 'extreme', '10082018', 'key', 'specie', 'criterion', 'yeah', 'given', 'stumble', 'einstein', 'il', 'bud', 'groundbreaking', 'saul', 'huge', 'jawdropping', 'rye', 'musk', 'diverse', 'sector', 'done', 'pan', 'solution', 'na', 'symbolizes', '230330', 'muscle', 'forest', 'ring', 'mignon', 'program', 'country', 'binbash', 'steamed', 'lightfilled', 'correlation', 'provide', 'lake', 'elegant', 'work', '144', 'lucky', 'tenderness', 'reconstruct', 'amazigh', 'conversation', 'calculate', 'equipment', 'mathematics', 'submodalities', 'personid', 'burnt', 'dea', 'connecting', 'httpswwwoverleafcomlatextemplatesocencfdarticletemplatemtpnnssgzjzw', 'terror', 'dautres', 'paperlined', 'offensive', 'access', 'distributed', 'manuel', 'payment', 'microsoft', 'hook', 'turkey', 'rival', 'disposition', 'runner', 'enforce'

In [170]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the 'processed_messages' column
tokenizer.fit_on_texts(df['processed_messages'])

# Convert the processed text into sequences of integers
sequences = tokenizer.texts_to_sequences(df['processed_messages'])

In [171]:
# Define the sequence length (n-gram size)
n_gram_length = 10  # You can adjust this for longer or shorter n-grams

# Create n-gram sequences
n_grams = []
for seq in sequences:
    for i in range(n_gram_length, len(seq)):
        n_gram_sequence = seq[i-n_gram_length:i+1]
        n_grams.append(n_gram_sequence)

# Convert the list of n-grams to a numpy array
n_grams = np.array(n_grams)

# Get the total number of unique words
total_unique_words = len(tokenizer.word_index) + 1

# Check the shape of the resulting n-grams
print(n_grams.shape)  # This will give you (number of sequences, n+1)

(7321, 11)


In [172]:
X = n_grams[:, :-1]
y = n_grams[:, -1]

In [173]:
# One-hot encode the target variable (y)
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_unique_words)

In [174]:
# Check the shape of X and y
print(X.shape)  # Input shape
print(y.shape)  # Output shape (one-hot encoded)

(7321, 10)
(7321, 4216)


In [175]:
print(f"Vocab size: {vocab_size}")
print(f"Shape of y: {y.shape}")

Vocab size: 4215
Shape of y: (7321, 4216)


In [176]:
X = np.clip(X, 0, vocab_size - 1)  # Clip values to be within the valid range (0 to 4214)

In [185]:
from keras.layers import Dropout

In [190]:
# Ensure correct vocab size based on unique words
vocab_size = 4215  # Number of unique words in the target

# Define the model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=n_gram_length,))
model.add(SimpleRNN(150, return_sequences=False))  
model.add(Dropout(0.3)) 
model.add(Dense(256, activation='relu'))
model.add(Dense(4216, activation='softmax'))  # Final layer should have 4215 units, not 4216
model.add(Dropout(0.3))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X, y, batch_size=64, epochs=50, verbose=1)


Epoch 1/50
115/115 [==============================] - 25s 55ms/step - loss: 10.3797 - accuracy: 0.0027
Epoch 2/50
115/115 [==============================] - 6s 56ms/step - loss: 10.0766 - accuracy: 0.0027
Epoch 3/50
115/115 [==============================] - 6s 55ms/step - loss: 9.8715 - accuracy: 0.0063
Epoch 4/50
115/115 [==============================] - 7s 57ms/step - loss: 9.7093 - accuracy: 0.0092
Epoch 5/50
115/115 [==============================] - 6s 56ms/step - loss: 9.4829 - accuracy: 0.0164
Epoch 6/50
115/115 [==============================] - 7s 57ms/step - loss: 9.1204 - accuracy: 0.0294
Epoch 7/50
115/115 [==============================] - 6s 56ms/step - loss: 8.7893 - accuracy: 0.0620
Epoch 8/50
115/115 [==============================] - 7s 58ms/step - loss: 8.2566 - accuracy: 0.1124
Epoch 9/50
115/115 [==============================] - 7s 58ms/step - loss: 7.7635 - accuracy: 0.1824
Epoch 10/50
115/115 [==============================] - 6s 56ms/step - loss: 7.2056 - acc

In [206]:
def generate_text(seed_text, next_words=10):
    for _ in range(next_words):
        # Tokenize the seed text
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        
        # Pad the sequence to the required length
        token_list = pad_sequences([token_list], maxlen=n_gram_length, padding='pre')
        
        # Predict the next word
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)[0]
        
        # Get the predicted word from the token index
        output_word = tokenizer.index_word.get(predicted)
        
        # If the predicted word is found in the index, add it to the seed text
        if output_word:
            seed_text += " " + output_word
        else:
            break  # Stop generating if no valid word is found
    
    return seed_text

# Generate new text
seed_text = "denote number"
print(generate_text(seed_text, next_words=10))


denote number aluminum aluminum hey chill river york city individual 6 exclusion


In [198]:
model2 = Sequential()
model2.add(keras.layers.Embedding(input_dim=vocab_size, output_dim=100, input_length=n_gram_length))

model2.add(keras.layers.Bidirectional(keras.layers.LSTM(100)))
model2.add(keras.layers.Dense(100, activation='relu'))
model2.add(keras.layers.Dense(4216, activation='softmax'))

In [199]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [200]:
model2.fit(X, y, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
229/229 [==============================] - 15s 41ms/step - loss: 8.2028 - accuracy: 0.0034
Epoch 2/10
229/229 [==============================] - 9s 41ms/step - loss: 7.7360 - accuracy: 0.0044
Epoch 3/10
229/229 [==============================] - 9s 40ms/step - loss: 7.4568 - accuracy: 0.0063
Epoch 4/10
229/229 [==============================] - 9s 40ms/step - loss: 7.1029 - accuracy: 0.0108
Epoch 5/10
229/229 [==============================] - 9s 40ms/step - loss: 6.6729 - accuracy: 0.0182
Epoch 6/10
229/229 [==============================] - 9s 39ms/step - loss: 6.1533 - accuracy: 0.0318
Epoch 7/10
229/229 [==============================] - 9s 41ms/step - loss: 5.5657 - accuracy: 0.0520
Epoch 8/10
229/229 [==============================] - 9s 39ms/step - loss: 4.9453 - accuracy: 0.0854
Epoch 9/10
229/229 [==============================] - 9s 40ms/step - loss: 4.3030 - accuracy: 0.1382
Epoch 10/10
229/229 [==============================] - 9s 40ms/step - loss: 3.6506 - accur

In [207]:
def generate_text(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=n_gram_length, padding='pre')
        predicted = np.argmax(model2.predict(token_list, verbose=0), axis=-1)[0]
        output_word = tokenizer.index_word.get(predicted, '')
        seed_text += " " + output_word
    return seed_text

# Generate new text
seed_text = "denote number"
print(generate_text(seed_text, next_words=10))

denote number silicone interviewer fine fine fine fine fine fine add add
